In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import joypy
from datetime import datetime
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import matplotlib.ticker as mtick


In [37]:
df = pd.read_csv("trippub.csv")
df = df[['HOUSEID','PERSONID','HHSTATE','TRPTRANS','TRPMILES','WTTRDFIN','TRAVDAY','TRIPPURP','HH_CBSA','STRTTIME']]

day = { 1 : 'Sunday', 2: 'Weekday', 3: 'Weekday', 4: 'Weekday', 5: "Weekday", 6: "Weekday", 7: "Saturday"}
day = pd.Series(day).to_frame('Day')
day = day.reset_index().rename(columns = {'index' : 'TRAVDAY'})

PURPOSE = { 'HBW' : "HBW", "HBO" : "HBO", "HBOSHOP": "HBO", "HBOSOCREC": "HBO", "NHB" : "NHB" }
PURPOSE = pd.Series(PURPOSE).to_frame('PURP')
PURPOSE = PURPOSE.reset_index().rename(columns = {'index' : 'TRIPPURP'})

df = pd.merge(df, day, on = 'TRAVDAY')
df = df.loc[(df['TRIPPURP'] != '-9'),]

df = pd.merge(df, PURPOSE, on = 'TRIPPURP')

df['TripDist'] = df['TRPMILES'] * df['WTTRDFIN']

df['Time'] = df['STRTTIME'].apply(lambda num : format(num,'04d'))
df['DateTime'] = df['Time'].apply(lambda num : datetime.strptime(num, "%H%M").strftime("%I:%M %p"))
df['Hour'] = pd.to_datetime(df['DateTime'], format='%I:%M %p')
df['NearestHour'] = df['Hour'].dt.round(freq='60min')


national_bike = df.loc[(df["TRPTRANS"] == 2),]
national_walk = df.loc[df["TRPTRANS"] == 1,]
weights_bike = national_bike['WTTRDFIN'].sum()

In [27]:
df.head()
df.dtypes

HOUSEID       int64
PERSONID      int64
HHSTATE      object
TRPTRANS      int64
TRPMILES    float64
WTTRDFIN    float64
TRAVDAY       int64
TRIPPURP     object
HH_CBSA      object
STRTTIME      int64
Day          object
PURP         object
TripDist    float64
dtype: object

In [39]:
# National Average Bike Trip Distance 

avg_dist_nat_bike = national_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']
avg_dist_nat_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.124916e+09,2.773284e+08,4.056261
Sunday,6.670968e+08,1.719340e+08,3.879959
Weekday,3.740563e+09,1.576857e+09,2.372164


In [40]:
# National Average Walk Trip Distance 

avg_dist_nat_walk = national_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']
avg_dist_nat_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.676141e+09,2.605913e+09,0.643207
Sunday,1.683127e+09,2.357477e+09,0.713953
Weekday,1.816740e+10,1.925264e+10,0.943632


In [41]:
# National Average Bike Trip Distance by Tour Purpose

avg_dist_nat_bike = national_bike.groupby(['Day','PURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


PURP,HBO,HBW,NHB
Day,,,
Saturday,2.635886,1.991578,5.113414
Sunday,5.871699,2.924829,3.208333
Weekday,2.213762,2.794740,2.068974


In [42]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['Day','PURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PURP,HBO,HBW,NHB
Day,,,
Saturday,0.817154,0.665986,0.513202
Sunday,0.901161,0.660000,0.530508
Weekday,0.885562,1.551666,0.894429


In [ ]:
#Time of Day

In [43]:
#Time of Day Bike National

avg_dist_nat_bike = national_bike.groupby(['NearestHour','PURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_nat_bike = pd.DataFrame(avg_dist_nat_bike)
avg_dist_nat_bike.reset_index()

avg_dist_nat_bike['AvgDist'] = avg_dist_nat_bike['TripDist']/avg_dist_nat_bike['WTTRDFIN']

avg_dist_nat_bike.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


PURP,HBO,HBW,NHB
NearestHour,,,
1900-01-01 00:00:00,NaN,0.977058,1.514000
1900-01-01 01:00:00,0.432000,0.993076,NaN
1900-01-01 02:00:00,NaN,2.321197,1.050384
1900-01-01 04:00:00,5.095175,1.191112,0.304000
1900-01-01 05:00:00,12.629936,1.091895,1.826577
1900-01-01 06:00:00,10.843460,4.338426,2.452367
1900-01-01 07:00:00,2.137506,2.870058,8.431268
1900-01-01 08:00:00,1.650946,2.763413,3.238734
1900-01-01 09:00:00,2.382128,2.641911,2.976190


In [44]:
# National Average Walk Trip Distance 
avg_dist_nat_walk = national_walk.groupby(['NearestHour','PURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_nat_walk = pd.DataFrame(avg_dist_nat_walk)
avg_dist_nat_walk.reset_index()

avg_dist_nat_walk['AvgDist'] = avg_dist_nat_walk['TripDist']/avg_dist_nat_walk['WTTRDFIN']

avg_dist_nat_walk.pivot_table(index = ['NearestHour'], values = 'AvgDist', columns = 'PURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PURP,HBO,HBW,NHB
NearestHour,,,
1900-01-01 00:00:00,0.665372,1.140169,0.254314
1900-01-01 01:00:00,0.331720,1.818587,3.941451
1900-01-01 02:00:00,0.897576,0.645482,1.410305
1900-01-01 03:00:00,0.356723,0.381000,0.505852
1900-01-01 04:00:00,2.420295,0.949275,0.641367
1900-01-01 05:00:00,0.855320,0.706411,0.892974
1900-01-01 06:00:00,1.067777,1.078918,0.769166
1900-01-01 07:00:00,0.764258,0.701864,12.139362
1900-01-01 08:00:00,1.383872,1.675016,1.040766


In [8]:
# FLORIDA

In [9]:
fl_bike = df.loc[(df["TRPTRANS"] == 2) & (df["HHSTATE"] == "FL"),]
fl_walk = df.loc[(df["TRPTRANS"] == 1) & (df["HHSTATE"] == "FL"),]

In [10]:
fl_bike.shape


(116, 11)

In [11]:
fl_walk.shape

(812, 11)

In [12]:
# Average Bike Trip Distance in Florida

avg_dist_fl_bike = fl_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']
avg_dist_fl_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,4.906882e+07,2.752865e+07,1.782464
Sunday,1.252354e+08,3.786933e+07,3.307041
Weekday,4.052103e+08,1.769850e+08,2.289517


In [13]:
#Average Walk Trip Distance in Florida

avg_dist_fl_walk = fl_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']
avg_dist_fl_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,1.038046e+08,1.989933e+08,0.521649
Sunday,1.333528e+08,2.905080e+08,0.459033
Weekday,2.423820e+09,1.185929e+09,2.043815


In [14]:
#Average Bike Trip Distance in Florida
avg_dist_fl_bike = fl_bike.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)
avg_dist_fl_bike.reset_index()

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']

avg_dist_fl_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')


C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,HBW,NHB
Day,,,,,
Saturday,2.299462,0.323000,1.660445,NaN,2.828000
Sunday,7.158275,0.636652,4.000844,6.359000,NaN
Weekday,1.916611,0.878860,1.868723,5.242045,1.449847


In [15]:
# Florida - Average Walk Trip Distance 
avg_dist_fl_walk = fl_walk.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk.reset_index()

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']

avg_dist_fl_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,HBW,NHB
Day,,,,,
Saturday,0.785272,0.604649,0.552377,0.236353,0.242131
Sunday,0.518640,0.315484,0.439120,0.258000,0.539781
Weekday,0.893879,5.220103,0.691502,26.116039,0.539827


In [17]:
fl_cbsa_bike = df.loc[(df["TRPTRANS"] == 2) & (df["HH_CBSA"] == '36740'),]
fl_cbsa_walk = df.loc[(df["TRPTRANS"] == 1) & (df["HH_CBSA"] == '36740'),]

In [18]:
fl_cbsa_bike.shape

(16, 11)

In [19]:
fl_cbsa_walk.shape

(75, 11)

In [20]:
########### CBSA ############

In [21]:
# FL CBSA - Average Bike Trip Distance

avg_dist_fl_bike = fl_cbsa_bike.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']
avg_dist_fl_bike

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Weekday,2.748425e+07,4.538354e+07,0.6056


In [22]:
# FL CBSA - Average Walk Trip Distance

avg_dist_fl_walk = fl_cbsa_walk.groupby('Day')['TripDist','WTTRDFIN'].sum()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']
avg_dist_fl_walk

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,TripDist,WTTRDFIN,AvgDist
Day,,,
Saturday,4.152402e+07,4.482420e+07,0.926375
Sunday,1.784226e+06,1.227661e+07,0.145335
Weekday,5.315958e+07,8.403644e+07,0.632578


In [23]:
# FL CBSA - Average Bike Trip Distance
avg_dist_fl_bike = fl_cbsa_bike.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].sum()
avg_dist_fl_bike = pd.DataFrame(avg_dist_fl_bike)
avg_dist_fl_bike.reset_index()

avg_dist_fl_bike['AvgDist'] = avg_dist_fl_bike['TripDist']/avg_dist_fl_bike['WTTRDFIN']

avg_dist_fl_bike.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')


C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC
Day,,,
Weekday,0.565725,0.570647,1.071965


In [24]:
# FL CBSA - Average Walk Trip Distance 
avg_dist_fl_walk = fl_cbsa_walk.groupby(['Day','TRIPPURP'])['TripDist','WTTRDFIN'].mean()
avg_dist_fl_walk = pd.DataFrame(avg_dist_fl_walk)
avg_dist_fl_walk.reset_index()

avg_dist_fl_walk['AvgDist'] = avg_dist_fl_walk['TripDist']/avg_dist_fl_walk['WTTRDFIN']

avg_dist_fl_walk.pivot_table(index = ['Day'], values = 'AvgDist', columns = 'TRIPPURP')

C:\Users\vyadav\AppData\Local\Continuum\anaconda3\envs\vy\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


TRIPPURP,HBO,HBSHOP,HBSOCREC,NHB
Day,,,,
Saturday,1.109104,1.4795,0.636407,0.219000
Sunday,NaN,NaN,0.087854,0.295250
Weekday,1.108155,1.0270,0.300567,0.362885
